In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import time
import shutil

In [ ]:
def extract_frames(video_path, output_folder, duration=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    end_frame = min(int(duration * video_fps), total_frames)
    frame_index = 0
    while cap.isOpened() and frame_index < end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        output_path = f"{output_folder}/frame_{frame_index:05d}.jpg"
        cv2.imwrite(output_path, frame)
        frame_index += 1
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
video_path = "input_video.mp4"
output_folder = "./output_frames"
duration = 30
extract_frames(video_path, output_folder, duration)

In [ ]:
def detect_faces_in_frames(frames_folder, output_folder):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    total_time = 0
    frame_count = 0
    for filename in os.listdir(frames_folder):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(frames_folder, filename)
            frame = cv2.imread(frame_path)
            start_time = time.time()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, frame)
            end_time = time.time()
            frame_processing_time = end_time - start_time
            total_time += frame_processing_time
            frame_count += 1
    average_time_per_frame = total_time / frame_count if frame_count > 0 else 0
    print("Average time per frame:", average_time_per_frame, "seconds")

In [ ]:
def detect_faces_in_frames_coloured(frames_folder, output_folder):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    total_time = 0
    frame_count = 0
    for filename in os.listdir(frames_folder):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(frames_folder, filename)
            frame = cv2.imread(frame_path)
            start_time = time.time()
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, frame)
            end_time = time.time()
            frame_processing_time = end_time - start_time
            total_time += frame_processing_time
            frame_count += 1
    average_time_per_frame = total_time / frame_count if frame_count > 0 else 0
    print("Average time per frame coloured:", average_time_per_frame, "seconds")

In [ ]:
frames_folder = "output_frames"
output_folder = "output_frames_with_faces"
output_folder_coloured = "output_frames_with_faces_coloured"
detect_faces_in_frames(frames_folder, output_folder)
detect_faces_in_frames_coloured(frames_folder, output_folder_coloured)

In [ ]:
def frames_to_video(frames_folder, output_video_path, fps=30):
    frame_files = [f for f in os.listdir(frames_folder) if f.endswith('.jpg')]
    frame_files.sort()
    frame_path = os.path.join(frames_folder, frame_files[0])
    frame = cv2.imread(frame_path)
    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    for filename in frame_files:
        frame_path = os.path.join(frames_folder, filename)
        frame = cv2.imread(frame_path)
        out.write(frame)
    out.release()
    print(f"Video saved successfully at '{output_video_path}'")

In [ ]:
frames_folder = "output_frames_with_faces"
output_video_path = "output_video.mp4"
frames_to_video(frames_folder, output_video_path)
frames_folder = "output_frames_with_faces_coloured"
output_video_path = "output_video_coloured.mp4"
frames_to_video(frames_folder, output_video_path)

In [ ]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    box1_area = w1 * h1
    box2_area = w2 * h2
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou